## How Many Pokémon Fit?

Finding the best **Pokémon** team by modeling and solving a knapsack problem with [PokeAPI](https://pokeapi.co/) and `PuLP` optimization Python library

In [1]:
!pip install -U pulp plotly==5.22.0

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/16.4 MB 825.8 kB/s eta 0:00:20
    --------------------------------------- 0.3/16.4 MB 2.5 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/16.4 MB 5.4 MB/s eta 0:00:03
   ----- ---------------------------------- 2.2/16.4 MB 10.8 MB/s eta 0:00:02
   ---------- ----------------------------- 4.3/16.4 MB 17.0 MB/s eta 0:00:01
   --------------- ------------------------ 6.2/16.4 MB 21.0 MB/s eta 0:00:01
   --------------------- ------------------ 8.7/16.4 MB 25.2 MB/s eta 0:00:01
   -------------------------- ------------- 10.9/16.4 MB 40.9 MB/s eta 0:00:01
   ------------------------------- -------- 12.9/16.4 MB 50.4 MB/s eta 0:00:01
   ------------------------------------- -- 15.4/16.4 MB 50.4 MB/s eta 0:00:01
   ---------------------------------------  16.4/16.4 MB 50.4 MB/s eta 0:00:01


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas as pd
import pulp

In [3]:
url = "https://pokeapi.co/api/v2/pokemon?limit=1500"
response = requests.get(url)
data = response.json()
pokemon_urls = [pokemon['url'] for pokemon in data['results']]

counter = 0
pokemon_data = []
for url in pokemon_urls:
    response = requests.get(url)
    pokemon = response.json()
    base_stats = {stat['stat']['name']: stat['base_stat'] for stat in pokemon['stats']}
    pokemon_data.append({
        'name': pokemon['name'],
        'weight': pokemon['weight'],
        'hp': base_stats.get('hp', 0),
        'attack': base_stats.get('attack', 0),
        'defense': base_stats.get('defense', 0),
        'special_attack': base_stats.get('special-attack', 0),
        'special_defense': base_stats.get('special-defense', 0),
        'speed': base_stats.get('speed', 0),
        'type': [t['type']['name'] for t in pokemon['types']],
    })
    counter += 1
    print("Adding {} -> {}".format(counter, pokemon['name']))

Adding 1 -> bulbasaur
Adding 2 -> ivysaur
Adding 3 -> venusaur
Adding 4 -> charmander
Adding 5 -> charmeleon
Adding 6 -> charizard
Adding 7 -> squirtle
Adding 8 -> wartortle
Adding 9 -> blastoise
Adding 10 -> caterpie
Adding 11 -> metapod
Adding 12 -> butterfree
Adding 13 -> weedle
Adding 14 -> kakuna
Adding 15 -> beedrill
Adding 16 -> pidgey
Adding 17 -> pidgeotto
Adding 18 -> pidgeot
Adding 19 -> rattata
Adding 20 -> raticate
Adding 21 -> spearow
Adding 22 -> fearow
Adding 23 -> ekans
Adding 24 -> arbok
Adding 25 -> pikachu
Adding 26 -> raichu
Adding 27 -> sandshrew
Adding 28 -> sandslash
Adding 29 -> nidoran-f
Adding 30 -> nidorina
Adding 31 -> nidoqueen
Adding 32 -> nidoran-m
Adding 33 -> nidorino
Adding 34 -> nidoking
Adding 35 -> clefairy
Adding 36 -> clefable
Adding 37 -> vulpix
Adding 38 -> ninetales
Adding 39 -> jigglypuff
Adding 40 -> wigglytuff
Adding 41 -> zubat
Adding 42 -> golbat
Adding 43 -> oddish
Adding 44 -> gloom
Adding 45 -> vileplume
Adding 46 -> paras
Adding 47 ->

In [4]:
df = pd.DataFrame(pokemon_data)

In [5]:
df

,name,weight,hp,attack,defense,special_attack,special_defense,speed,type
0,bulbasaur,69,45,49,49,65,65,45,"[grass, poison]"
1,ivysaur,130,60,62,63,80,80,60,"[grass, poison]"
2,venusaur,1000,80,82,83,100,100,80,"[grass, poison]"
3,charmander,85,39,52,43,60,50,65,[fire]
4,charmeleon,190,58,64,58,80,65,80,[fire]
...,...,...,...,...,...,...,...,...,...
1297,ogerpon-wellspring-mask,398,80,120,84,60,96,110,"[grass, water]"
1298,ogerpon-hearthflame-mask,398,80,120,84,60,96,110,"[grass, fire]"
1299,ogerpon-cornerstone-mask,398,80,120,84,60,96,110,"[grass, rock]"
1300,terapagos-terastal,160,95,95,110,105,110,85,[normal]


In [7]:
prob = pulp.LpProblem("Pokemon-Team-Optimization", pulp.LpMaximize)

In [8]:
x = pulp.LpVariable.dicts("x", range(len(pokemon_data)), cat='Binary')

In [9]:
prob += pulp.lpSum(
    (pokemon['hp'] + pokemon['attack'] + pokemon['defense'] + pokemon['special_attack'] + pokemon['special_defense'] + 
     pokemon['speed']) * x[i] for i, pokemon in enumerate(pokemon_data)
), "Total Combat Effectiveness"

In [10]:
# total weight constraint
max_weight_capacity = 1000
prob += pulp.lpSum(pokemon['weight'] * x[i] for i, pokemon in enumerate(pokemon_data)) <= max_weight_capacity, "Weight Capacity"

# total number of Pokemon constraint
prob += pulp.lpSum(x[i] for i in range(len(pokemon_data))) == 6, "Team Size"

In [11]:
prob.solve()

1

In [12]:
selected_pokemon = [pokemon_data[i]['name'] for i in range(len(pokemon_data)) if pulp.value(x[i]) == 1]
print("Selected Pokémon:", selected_pokemon)

Selected Pokémon: ['jirachi', 'meloetta-aria', 'pecharunt', 'mewtwo-mega-y', 'diancie-mega', 'eternatus-eternamax']


In [14]:
# Calculate BST for all Pokémon
for pokemon in pokemon_data:
    pokemon['BST'] = (pokemon['hp'] + pokemon['attack'] + pokemon['defense'] +
                                             pokemon['special_attack'] + pokemon['special_defense'] + 
                                             pokemon['speed'])
    
all_pokemon_df = pd.DataFrame(pokemon_data)
selected_df = pd.DataFrame(selected_pokemon)

# Create the scatter plot
import plotly.graph_objects as go
fig = go.Figure()

# Add all Pokémon
fig.add_trace(go.Scatter(
    x=all_pokemon_df['weight'],
    y=all_pokemon_df['BST'],
    mode='markers',
    name='All Pokémon',
    marker=dict(size=10, color='blue', opacity=0.6),
    text=all_pokemon_df['name']
))

# Add selected Pokémon
fig.add_trace(go.Scatter(
    x=selected_df['weight'],
    y=selected_df['total_combat_effectiveness'],
    mode='markers',
    name='Selected Pokémon',
    marker=dict(size=12, color='red', opacity=0.9),
    text=selected_df['name']
))

# Add title and labels, resize
fig.update_layout(
    title="Pokémon Selection Optimization",
    xaxis_title="Weight (KGs)",
    yaxis_title="BST",
    legend_title="Legend",
    width=1000, 
    height=600,
    showlegend=True
)

# Show the plot
fig.show()

KeyError: 'weight'